In [1]:
import math
import torch
import string
import torch.nn as nn

from fairseq.models.bart import BARTModel
from utils import read_lines, get_probability

ModuleNotFoundError: No module named 'fairseq'

In [ ]:
bart = BARTModel.from_pretrained('/home/ml/cadencao/Downloads/BART_models/bart.large.xsum',
                                 checkpoint_file='model.pt',
                                 data_name_or_path='/home/ml/cadencao/Downloads/BART_models/bart.large.xsum')

In [ ]:
bart.cuda()
bart.eval()
bart.half()
print('- activate evaluation mode')

In [ ]:
encode_func = bart.encode
decode_func = bart.decode

In [ ]:
bart_encoder = bart.model.encoder
bart_decoder = bart.model.decoder
print(type(bart.model))
print(type(bart_encoder))
print(type(bart_decoder))

#### Read XSum

In [ ]:
document_path = '/home/ml/cadencao/XSum/fairseq_files/test.source'
target_path = '/home/ml/cadencao/XSum/fairseq_files/test.target'
xsum_source = read_lines(document_path)
xsum_target = read_lines(target_path)
print(len(xsum_source))
assert len(xsum_source) == len(xsum_target)

#### Tokenization

In [ ]:
import spacy
import rouge

from fairseq.data.data_utils import collate_tokens

In [ ]:
def prepare_results(p, r, f):
    return '\t{}:\t{}: {:5.2f}\t{}: {:5.2f}\t{}: {:5.2f}'.format(metric, 'P', 100.0 * p, 'R', 100.0 * r, 'F1', 100.0 * f)

aggregator = 'Avg'
apply_avg = aggregator == 'Avg'
apply_best = aggregator == 'Best'
print('Evaluation with {}'.format(aggregator))

evaluator = rouge.Rouge(metrics=['rouge-n', 'rouge-l', 'rouge-w'],
                        max_n=4,
                        limit_length=True,
                        length_limit=100,
                        length_limit_type='words',
                        apply_avg=apply_avg,
                        apply_best=apply_best,
                        alpha=0.5, # Default F1_score
                        weight_factor=1.2,
                        stemming=True)

In [ ]:
nlp = spacy.load('en')

In [ ]:
INDEX = 10579
test_inputs = [xsum_source[INDEX]]
src_tokens = collate_tokens([encode_func(i) for i in test_inputs], pad_idx=1, left_pad=True)
src_tokens = src_tokens.cuda()
print(src_tokens.shape)

src_lengths = torch.sum(src_tokens != 1, dim=1)
print(src_lengths)

In [ ]:
# encoding

In [ ]:
encoder_out = bart_encoder(src_tokens, src_lengths=src_lengths)
print(encoder_out.encoder_out.shape)  # [seq_len, batch_size, hidden_size]

In [ ]:
# decoding

In [ ]:
init_input = torch.tensor([[2, 0]] * src_tokens.shape[0], dtype=torch.long).cuda()
min_decode_step, max_decode_step, pad_id, eos_id = 10, 60, 1, 2
softmax = nn.Softmax(dim=1)
token_probs, tokens = [], []

for step in range(max_decode_step):
    decoder_outputs = bart_decoder(init_input, encoder_out, features_only=False)
    logits = decoder_outputs[0][:, -1, :]  # [batch_size, vocab]
    if step + 1 < min_decode_step:
        logits[:, eos_id] = -math.inf

    logits[:, pad_id], logits[:, 0] = -math.inf, -math.inf  # never select pad, start token
    probs = softmax(logits)
    assert logits.shape == probs.shape
    attn = decoder_outputs[1]['attn'][0]  # [batch_size, prev_token_len, src_token_len]
    assert logits.dim() == 2 and attn.dim() == 3

    # preds = torch.argmax(logits, dim=1)
    value, indices = torch.topk(probs, 5, dim=1)
    selected_token = indices[:, 0]

    if step == 1:
        selected_token = indices[:, 0]
    elif step == 10:
        selected_token = indices[:, 0]
    elif step == 12:
        selected_token = indices[:, 0]

    init_input = torch.cat([init_input, selected_token.unsqueeze(1)], dim=-1)
    token, prob = decode_func(selected_token), probs.squeeze()[selected_token.item()].item()
    token_probs.append(prob)
    tokens.append(token)

    if selected_token.item() == eos_id:
        break
    else:
        print("- {:02d}: {} ({:.2f})".format(step, token, prob), end='\n')

assert len(tokens) == len(token_probs)
token_probs_sorted = [i for i in token_probs]
token_probs_sorted.sort()
print('- Lowest Probabilities: [{}, {}, {}]'.format(token_probs.index(token_probs_sorted[0]),
                           token_probs.index(token_probs_sorted[1]),
                           token_probs.index(token_probs_sorted[2])))

In [ ]:
get_probability('Ryan Chase', tokens, token_probs)

In [ ]:
init_input

In [ ]:
hypothesis = decode_func(init_input[0])
print(hypothesis)

In [ ]:
nlp(hypothesis).ents

In [ ]:
xsum_target[INDEX]

In [ ]:
all_hypothesis = [hypothesis]
all_references = [xsum_target[INDEX]]
scores = evaluator.get_scores(all_hypothesis, all_references)

for metric, results in sorted(scores.items(), key=lambda x: x[0]):
    print(prepare_results(results['p'], results['r'], results['f']))

In [ ]:
xsum_source[INDEX]